# Fake Vs Real Image Detection

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 1197 images belonging to 2 classes.


### Preprocessing the Test set

In [3]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 512 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [4]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [5]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [6]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [8]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [9]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [10]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [11]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [12]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
38/38 [==============================] - 28s 704ms/step - loss: 0.7173 - accuracy: 0.5522 - val_loss: 0.6883 - val_accuracy: 0.5547
Epoch 2/10
38/38 [==============================] - 27s 723ms/step - loss: 0.6868 - accuracy: 0.5556 - val_loss: 0.6849 - val_accuracy: 0.5547
Epoch 3/10
38/38 [==============================] - 28s 734ms/step - loss: 0.6816 - accuracy: 0.5564 - val_loss: 0.6769 - val_accuracy: 0.5879
Epoch 4/10
38/38 [==============================] - 26s 699ms/step - loss: 0.6701 - accuracy: 0.5998 - val_loss: 0.6640 - val_accuracy: 0.6094
Epoch 5/10
38/38 [==============================] - 27s 704ms/step - loss: 0.6652 - accuracy: 0.5998 - val_loss: 0.6713 - val_accuracy: 0.5762
Epoch 6/10
38/38 [==============================] - 27s 712ms/step - loss: 0.6672 - accuracy: 0.6048 - val_loss: 0.6636 - val_accuracy: 0.6035
Epoch 7/10
38/38 [==============================] - 27s 710ms/step - loss: 0.6688 - accuracy: 0.5940 - val_loss: 0.6757 - val_accuracy: 0.5605

## Part 4 - Making a single prediction

In [44]:
test_final_datagen = ImageDataGenerator(rescale = 1./255)
test_final_set = test_final_datagen.flow_from_directory('test_final',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 332 images belonging to 1 classes.


In [45]:
pred = pd.DataFrame(cnn.predict(test_final_set))

<IPython.core.display.Javascript object>

In [46]:
pred=pred.round()
pred.columns= ['label']
pred.head()

,label
0,0.0
1,1.0
2,0.0
3,0.0
4,1.0


In [47]:
testing_data= pd.read_csv('test.csv')
testing_data.head()

<IPython.core.display.Javascript object>

,file_id
0,0
1,1
2,2
3,3
4,4


In [48]:
result=pd.concat([testing_data, pred], join = 'outer', axis = 1)
result.head()

<IPython.core.display.Javascript object>

,file_id,label
0,0,0.0
1,1,1.0
2,2,0.0
3,3,0.0
4,4,1.0


In [49]:
result.to_csv('result4.csv')

In [50]:
result['label'].value_counts()

1.0    179
0.0    153
Name: label, dtype: int64